### Default to Iteration 1: Above-the-Line / Below-the-Line (ATL/BTL) Testing

In [1]:
import os; 
import pandas as pd; pd.set_option('mode.chained_assignment',None)
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import re
import xlrd
import statistics as stat
from xlsxwriter.utility import xl_rowcol_to_cell
from random import sample
from src.data_models.SmartsCsvDataModel import SmartsCsvDataModel
from src.data_models.smartshelper import metrics, fixNum, unSMART, cparty
from src.data_models.SmartsDataModel import SmartsDataModel
from src.utility.DataModelUtility import execute_query_data_frame
from src.data_models.SmartsDataModel import SmartsDataModel
from src.data_models.UPMhelper import UPMalerts

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### Step 0) Lookback Days {20,30,40}

In [2]:
step0 = unSMART(pd.read_excel('~/Documents/vmfldr/Alerts-AllRows_gemini_TEST1.xlsx'))
step1 = unSMART(pd.read_excel('~/Documents/vmfldr/Alerts-AllRows_gemini-step1.xlsx'))
step2 = unSMART(pd.read_excel('~/Documents/vmfldr/Alerts-AllRows_gemini-step2.xlsx'))
step3 = unSMART(pd.read_excel('~/Documents/vmfldr/Alerts-AllRows_gemini-step3.xlsx'))
step4 = unSMART(pd.read_excel('~/Documents/vmfldr/Alerts-AllRows_gemini-step4.xlsx'))

-> type fixed, kept 36884 rows, 13 columns
-> type fixed, kept 28349 rows, 13 columns
-> type fixed, kept 2863 rows, 13 columns
-> type fixed, kept 1908 rows, 13 columns
-> type fixed, kept 1801 rows, 13 columns


In [3]:
step0['ShortText3'] = step0['ShortText'].str.replace('\(MC\) ','').str.replace('\(RP\) ','')
step1['ShortText3'] = step1['ShortText'].str.replace('\(MC\) ','').str.replace('\(RP\) ','')
step2['ShortText3'] = step2['ShortText'].str.replace('\(MC\) ','').str.replace('\(RP\) ','')
step3['ShortText3'] = step3['ShortText'].str.replace('\(MC\) ','').str.replace('\(RP\) ','')
step4['ShortText3'] = step4['ShortText'].str.replace('\(MC\) ','').str.replace('\(RP\) ','')

In [4]:
step0_1 = step0.loc[step0['Datetime'].dt.month == 12]
step1_1 = step1.loc[step1['Datetime'].dt.month == 12]
step2_1 = step2.loc[step2['Datetime'].dt.month == 12]
step3_1 = step3.loc[step3['Datetime'].dt.month == 12]
step4_1 = step4.loc[step4['Datetime'].dt.month == 12]

print(step0.shape)
print(step1.shape)
print(step2.shape)
print(step3.shape)
print(step4.shape)

(36884, 14)
(28349, 14)
(2863, 14)
(1908, 14)
(1801, 14)


In [29]:
step4_1.loc[step4_1['AlertCode'].isin([4041,4042])].groupby(['AlertCode','Date']).count().count()

Time              28
InstrumentName    28
ShortText         28
AccountIDName     28
ReissueCount      28
Reissued          28
LongText          28
Datetime          28
month             28
year              28
ShortText_mod     28
ShortText3        28
dtype: int64

In [5]:
step0_2 = pd.DataFrame(step0_1.groupby(['AlertCode', 'ShortText3']).count()['Reissued']).rename(columns={'Reissued':'step0'})
step1_2 = pd.DataFrame(step1_1.groupby(['AlertCode', 'ShortText3']).count()['Reissued']).rename(columns={'Reissued':'step1'})
step2_2 = pd.DataFrame(step2_1.groupby(['AlertCode', 'ShortText3']).count()['Reissued']).rename(columns={'Reissued':'step2'})
step3_2 = pd.DataFrame(step3_1.groupby(['AlertCode', 'ShortText3']).count()['Reissued']).rename(columns={'Reissued':'step3'})
step4_2 = pd.DataFrame(step4_1.groupby(['AlertCode', 'ShortText3']).count()['Reissued']).rename(columns={'Reissued':'step4'})

In [6]:
lookback_impact = pd.concat([step0_2,step1_2,step2_2,step3_2,step4_2], axis=1)

In [20]:
lookback_impact['steps0_1%'] = round(lookback_impact['step1']/lookback_impact['step0'] - 1,2) * 100
lookback_impact['steps1_2%'] = round(lookback_impact['step2']/lookback_impact['step1'] - 1,2) * 100
lookback_impact['steps2_3%'] = round(lookback_impact['step3']/lookback_impact['step2'] - 1,2) * 100
lookback_impact['steps3_4%'] = round(lookback_impact['step4']/lookback_impact['step3'] - 1,2) * 100
lookback_impact.astype(int)#.to_csv('~/Desktop/outputtemp1.csv')

step0  step1  step2  step3  step4  \
AlertCode ShortText3                                                            
1001      UNUSUAL PRICE MOVEMENT INTRA-DAY     98    124    124    124    124   
4009      PAINTING THE TAPE                    39     57     57     57     57   
4012      PRICE DRIVER                         32      4     13     13     13   
4022      BAIT AND SWITCH                   14816  10427     62     38     36   
4032      MULTI ORDER SPOOFING                433    468    293     41     12   
          MULTI ORDER SPOOFING EOD            415    443    284     41     29   
4041      WASH SALE A-B-A                      28     30      4      4      4   
4042      WASH SALE A-B-A (MONEY PASS)       2702   2145    296    296    296   
4045      COLLUSION(MONEY PASS)                 3      5      3      3      3   

                                            steps0_1%  steps1_2%  steps2_3%  \
AlertCode ShortText3                                                          
1001      UNUSUAL PRICE MOVEMENT INTRA-DAY         27          0          0   
4009      PAINTING THE TAPE                        46          0          0   
4012      PRICE DRIVER                            -88        225          0   
4022      BAIT AND SWITCH                         -30        -99        -39   
4032      MULTI ORDER SPOOFING                      8        -37        -86   
          MULTI ORDER SPOOFING EOD                  7        -36        -86   
4041      WASH SALE A-B-A                           7        -87          0   
4042      WASH SALE A-B-A (MONEY PASS)            -21        -86          0   
4045      COLLUSION(MONEY PASS)                    67        -40          0   

                                            steps3_4%  
AlertCode ShortText3                                   
1001      UNUSUAL PRICE MOVEMENT INTRA-DAY          0  
4009      PAINTING THE TAPE                         0  
4012      PRICE DRIVER                              0  
4022      BAIT AND SWITCH                          -5  
4032      MULTI ORDER SPOOFING                    -71  
          MULTI ORDER SPOOFING EOD                -28  
4041      WASH SALE A-B-A                           0  
4042      WASH SALE A-B-A (MONEY PASS)              0  
4045      COLLUSION(MONEY PASS)                     0

### Values -1001

In [8]:
# regextext1 = '\D+(Price Change.+)Bid.+'
# regextext2 = 'Price Change\s(?:in the last )?(\d+\D+|trade to trade)\sis\D+([\d.]+)\D+([\d.]+)\D+[\d.]+\D+[\d.]+\D+([\d.]+)\D+([\d.]+)%\)\s+'

# alerts1001 = lookback_30m.loc[lookback_30m['AlertCode'] == 1001,['AlertCode','LongText','InstrumentName']]
# alerts1001['LongText2'] = alerts1001['LongText'].str.extract(regextext1,expand=True)
# alerts1001_rematch = alerts1001['LongText2'].str.extractall(regextext2).reset_index()
# alerts1001_rematch.columns = ['index','index2','range','valdiff','valperc','bmdiff','bmperc']
# alerts1001_rematch['symbol'] = alerts1001_rematch.reset_index()['index'].map(alerts1001['InstrumentName'].to_dict())
# alerts1001_rematch.groupby('range')['valperc'].count()

In [9]:
# regextext1 = '\D+(Price Change.+)Bid.+'
# regextext2 = 'Price Change\s(?:in the last )?(\d+\D+|trade to trade)\sis\D+([\d.]+)\D+([\d.]+)\D+[\d.]+\D+[\d.]+\D+([\d.]+)\D+([\d.]+)%\)\s+'

# alerts1001 = lookback_prod.loc[lookback_prod['AlertCode'] == 1001,['AlertCode','LongText','InstrumentName']]
# alerts1001['LongText2'] = alerts1001['LongText'].str.extract(regextext1,expand=True)
# alerts1001_rematch = alerts1001['LongText2'].str.extractall(regextext2).reset_index()
# alerts1001_rematch.columns = ['index','index2','range','valdiff','valperc','bmdiff','bmperc']
# alerts1001_rematch['symbol'] = alerts1001_rematch.reset_index()['index'].map(alerts1001['InstrumentName'].to_dict())
# alerts1001_rematch.groupby('range')['valperc'].count()

### Values -2011

In [10]:
# alerts2011 = lookback_prodm.loc[lookback_prodm['AlertCode'] == 2011,['AlertCode','LongText','InstrumentName']]
# regextext3 = '[^:]*\:*\s*([A-Z]{6})\D+([\d,.]+)\D+(\d+\D+)\.\D+([\d,.]+)\s*'
# alerts2011_rematch = alerts2011['LongText'].str.extractall(regextext3).reset_index()
# alerts2011_rematch.columns = ['index','index2','symbol','value','range','bm']
# alerts2011_rematch.groupby('range')['value'].count()

### Values -4012

In [11]:
lookback = step0_1
alerts4012 = lookback.loc[lookback['AlertCode'] == 4012,['AlertCode','LongText','InstrumentName']]
regextext3 = '\D+(\d+)\D+([A-Z]{6})\D+(\d+)\D+(\d+)[^\(]+\(([\d.,]+)%[^\(]+\(([\d.,]+)%.+'
alerts4012_rematch = alerts4012['LongText'].str.extract(regextext3); alerts4012_rematch
alerts4012_rematch.columns = ['acc','symbol','num','denom','tick','vol']
alerts4012_rematch[['tick','vol']] = alerts4012_rematch[['tick','vol']].astype(float)
alerts4012_rematch['dir'] = alerts4012_rematch['num'].astype(float)/alerts4012_rematch['denom'].astype(float)
step0output = alerts4012_rematch[['dir','tick','vol']]#.to_csv('~/Desktop/tuned_pdinvestigation.csv')

In [12]:
lookback = step1_1
alerts4012 = lookback.loc[lookback['AlertCode'] == 4012,['AlertCode','LongText','InstrumentName']]
regextext3 = '\D+(\d+)\D+([A-Z]{6})\D+(\d+)\D+(\d+)[^\(]+\(([\d.,]+)%[^\(]+\(([\d.,]+)%.+'
alerts4012_rematch = alerts4012['LongText'].str.extract(regextext3); alerts4012_rematch
alerts4012_rematch.columns = ['acc','symbol','num','denom','tick','vol']
alerts4012_rematch[['tick','vol']] = alerts4012_rematch[['tick','vol']].astype(float)
alerts4012_rematch['dir'] = alerts4012_rematch['num'].astype(float)/alerts4012_rematch['denom'].astype(float)
step1output = alerts4012_rematch[['dir','tick','vol']]#.to_csv('~/Desktop/tuned_pdinvestigation.csv')

In [13]:
lookback = step2_1
alerts4012 = lookback.loc[lookback['AlertCode'] == 4012,['AlertCode','LongText','InstrumentName']]
regextext3 = '\D+(\d+)\D+([A-Z]{6})\D+(\d+)\D+(\d+)[^\(]+\(([\d.,]+)%[^\(]+\(([\d.,]+)%.+'
alerts4012_rematch = alerts4012['LongText'].str.extract(regextext3); alerts4012_rematch
alerts4012_rematch.columns = ['acc','symbol','num','denom','tick','vol']
alerts4012_rematch[['tick','vol']] = alerts4012_rematch[['tick','vol']].astype(float)
alerts4012_rematch['dir'] = alerts4012_rematch['num'].astype(float)/alerts4012_rematch['denom'].astype(float)
step2output = alerts4012_rematch[['dir','tick','vol']]#.to_csv('~/Desktop/tuned_pdinvestigation.csv')

In [14]:
lookback = step3_1
alerts4012 = lookback.loc[lookback['AlertCode'] == 4012,['AlertCode','LongText','InstrumentName']]
regextext3 = '\D+(\d+)\D+([A-Z]{6})\D+(\d+)\D+(\d+)[^\(]+\(([\d.,]+)%[^\(]+\(([\d.,]+)%.+'
alerts4012_rematch = alerts4012['LongText'].str.extract(regextext3)
alerts4012_rematch.columns = ['acc','symbol','num','denom','tick','vol']
alerts4012_rematch[['tick','vol']] = alerts4012_rematch[['tick','vol']].astype(float)
alerts4012_rematch['dir'] = alerts4012_rematch['num'].astype(float)/alerts4012_rematch['denom'].astype(float)
step3output = alerts4012_rematch[['dir','tick','vol']]#.to_csv('~/Desktop/tuned_pdinvestigation.csv')

In [15]:
lookback = step4_1
alerts4012 = lookback.loc[lookback['AlertCode'] == 4012,['AlertCode','LongText','InstrumentName']]
regextext3 = '\D+(\d+)\D+([A-Z]{6})\D+(\d+)\D+(\d+)[^\(]+\(([\d.,]+)%[^\(]+\(([\d.,]+)%.+'
alerts4012_rematch = alerts4012['LongText'].str.extract(regextext3); alerts4012_rematch
alerts4012_rematch.columns = ['acc','symbol','num','denom','tick','vol']
alerts4012_rematch[['tick','vol']] = alerts4012_rematch[['tick','vol']].astype(float)
alerts4012_rematch['dir'] = alerts4012_rematch['num'].astype(float)/alerts4012_rematch['denom'].astype(float)
step4output = alerts4012_rematch[['dir','tick','vol']]#.to_csv('~/Desktop/tuned_pdinvestigation.csv')

In [18]:
os.chdir('/Users/sqian/Documents')

writer = pd.ExcelWriter('PDoutput.xlsx', engine='xlsxwriter')

step0output.to_excel(writer, 'step0output')
step1output.to_excel(writer, 'step1output')
step2output.to_excel(writer, 'step2output')
step3output.to_excel(writer, 'step3output')
step4output.to_excel(writer, 'step4output')

writer.save()